# 12.1　自动生成XSS攻击载荷

## 12.1.1　数据集

In [1]:
import os
import pickle
from six.moves import urllib
import tflearn
from tflearn.data_utils import *

char_idx_file = 'char_idx_xss.pkl'
maxlen = 25
char_idx = None
xss_data_file = "../Data/aiscanner/xss.txt"

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


## 12.1.2　特征提取

In [2]:
if os.path.isfile(char_idx_file):
    print('Loading previous xss_char_idx')
    char_idx = pickle.load(open(char_idx_file, 'rb'))

X, Y, char_idx = textfile_to_semi_redundant_sequences(xss_data_file,
                        seq_maxlen=maxlen, # 序列的最大长度
                        redun_step=3,
                        pre_defined_char_idx=char_idx) # 如果使用指定的字符和数字的转换关系，需要设置该参数，如果设置为None，将根据文本自动生成char_idx结构

Vectorizing text...
Text total length: 104,227
Distinct chars   : 110
Total sequences  : 34,734


处理结果如下所示，完整的字符个数为104269，字符种类为116，对应的字符序列个数为34748：

## 12.1.3　模型训练与验证

使用RNN的LSTM算法，特征提取字符序列。

1. 读取XSS攻击载荷数据集数据。
2. 转换成字符序列。
3. 使用LSTM算法在数据集上训练。
4. 从数据集随机生成种子。
5. LSTM算法使用种子生成新的XSS攻击载荷。

LSTM的输入的张量是二维的，大小为（maxlen，len（char_idx）​）​，其中行为字符序列的最大长度，列为char_idx的长度：

In [3]:
g = tflearn.input_data([None, maxlen, len(char_idx)])
g = tflearn.lstm(g, 32, return_seq=True)
g = tflearn.dropout(g, 0.1)
g = tflearn.lstm(g, 32, return_seq=True)
g = tflearn.dropout(g, 0.1)
g = tflearn.lstm(g, 32)
g = tflearn.dropout(g, 0.1)
g = tflearn.fully_connected(g, len(char_idx), activation='softmax')
g = tflearn.regression(g, optimizer='adam', loss='categorical_crossentropy',
                        learning_rate=0.001)
g

Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


<tf.Tensor 'FullyConnected/Softmax:0' shape=(?, 110) dtype=float32>

In [4]:
m = tflearn.SequenceGenerator(g, dictionary=char_idx,
                                seq_maxlen=maxlen,
                                clip_gradients=5.0,
                                checkpoint_path='chkpoint/model_scanner_poc')

In [5]:

print("random_sequence_from_textfile")
# seed = '"/><script>'
seed = random_sequence_from_textfile(xss_data_file,maxlen)
m.fit(X, Y, validation_set=0.1, batch_size=128, n_epoch=2, run_id='scanner-poc')

Training Step: 489  | total loss: 4.08868 | time: 7.955s
| Adam | epoch: 002 | loss: 4.08868 -- iter: 31232/31260
Training Step: 490  | total loss: 4.08175 | time: 9.004s
| Adam | epoch: 002 | loss: 4.08175 | val_loss: 3.96259 -- iter: 31260/31260
--
INFO:tensorflow:d:\Documents\天池竞赛\WebSecMLLearnNote\Code2\chkpoint\model_scanner_poc-490 is not in all_model_checkpoint_paths. Manually adding it.


基于种子，不断生成新的序列，即XSS的攻击载荷，其中temperature可以理解为新颖程度，temperature越小，生成的序列就越接近原有序列，反之亦然：

In [6]:
print("-- TESTING...")
seed = random_sequence_from_textfile(xss_data_file,maxlen)
print("-- Test with temperature of 0.1 --")
m.generate(32, temperature=0.1, seq_seed=seed)

-- TESTING...
-- Test with temperature of 0.1 --


't:alert(1)"></svg onLoad>eeataraaererraaeataraerasarratrt'

In [7]:
print("-- Test with temperature of 0.5 --")
print(m.generate(32, temperature=0.5, seq_seed=seed))

-- Test with temperature of 0.5 --
t:alert(1)"></svg onLoad>ara"prctaa.raappts
aruoboEapseas


In [8]:
print("-- Test with temperature of 1.0 --")
print(m.generate(32, temperature=1.0, seq_seed=seed))

-- Test with temperature of 1.0 --
t:alert(1)"></svg onLoad>selg3tcy&&rntT18yu Ftnb.>tserv= 


实际环境下，种子往往可以手工指定，通过指定种子，可以按照需求生成XSS攻击载荷

In [9]:
seed = '"/><script>'
m.generate(32, temperature=1.0, seq_seed=seed)

'"/><script>"jrI&jeBc=m/apu7\n>;arRa\'Screlvx='

# 12.2　自动识别登录界面

## 12.2.1　数据集

In [10]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train')
from pprint import pprint

In [11]:
pprint(list(newsgroups_train.target_names))

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']


In [12]:
cats = ['alt.atheism', 'sci.space']
newsgroups_train = fetch_20newsgroups(subset='train', categories=cats)

In [13]:
newsgroups_train.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [14]:
print(newsgroups_train['DESCR'])

.. _20newsgroups_dataset:

The 20 newsgroups text dataset
------------------------------

The 20 newsgroups dataset comprises around 18000 newsgroups posts on
20 topics split in two subsets: one for training (or development)
and the other one for testing (or for performance evaluation). The split
between the train and test set is based upon a messages posted before
and after a specific date.

This module contains two loaders. The first one,
:func:`sklearn.datasets.fetch_20newsgroups`,
returns a list of the raw texts that can be fed to text feature
extractors such as :class:`~sklearn.feature_extraction.text.CountVectorizer`
with custom parameters so as to extract feature vectors.
The second one, :func:`sklearn.datasets.fetch_20newsgroups_vectorized`,
returns ready-to-use features, i.e., it is not necessary to use a feature
extractor.

**Data Set Characteristics:**

    =================   ==========
    Classes                     20
    Samples total            18846
    Dimensionality

In [15]:
import pandas as pd

nt = pd.DataFrame({"data":newsgroups_train['data'],
                   "filenames":newsgroups_train['filenames'],
                   "target":newsgroups_train['target']})
nt

,data,filenames,target
0,From: bil@okcforum.osrhe.edu (Bill Conner)\nSu...,D:\scikit_learn_data\20news_home\20news-bydate...,0
1,From: jhwitten@cs.ruu.nl (Jurriaan Wittenberg)...,D:\scikit_learn_data\20news_home\20news-bydate...,1
2,From: sysmgr@king.eng.umd.edu (Doug Mohney)\nS...,D:\scikit_learn_data\20news_home\20news-bydate...,1
3,From: pgf@srl03.cacs.usl.edu (Phil G. Fraering...,D:\scikit_learn_data\20news_home\20news-bydate...,1
4,From: Nanci Ann Miller <nm0w+@andrew.cmu.edu>\...,D:\scikit_learn_data\20news_home\20news-bydate...,0
...,...,...,...
1068,From: sysmgr@king.eng.umd.edu (Doug Mohney)\nS...,D:\scikit_learn_data\20news_home\20news-bydate...,1
1069,From: kjenks@jsc.nasa.gov (Ken Jenks [NASA])\n...,D:\scikit_learn_data\20news_home\20news-bydate...,1
1070,From: nicho@vnet.IBM.COM (Greg Stewart-Nicholl...,D:\scikit_learn_data\20news_home\20news-bydate...,1
1071,From: eder@hsvaic.boeing.com (Dani Eder)\nSubj...,D:\scikit_learn_data\20news_home\20news-bydate...,1


In [16]:
print(nt.iloc[0,1])
print(nt.iloc[0,0])

D:\scikit_learn_data\20news_home\20news-bydate-train\alt.atheism\51312
From: bil@okcforum.osrhe.edu (Bill Conner)
Subject: Re: Not the Omni!
Nntp-Posting-Host: okcforum.osrhe.edu
Organization: Okcforum Unix Users Group
X-Newsreader: TIN [version 1.1 PL6]
Lines: 18

Charley Wingate (mangoe@cs.umd.edu) wrote:
: 
: >> Please enlighten me.  How is omnipotence contradictory?
: 
: >By definition, all that can occur in the universe is governed by the rules
: >of nature. Thus god cannot break them. Anything that god does must be allowed
: >in the rules somewhere. Therefore, omnipotence CANNOT exist! It contradicts
: >the rules of nature.
: 
: Obviously, an omnipotent god can change the rules.

When you say, "By definition", what exactly is being defined;
certainly not omnipotence. You seem to be saying that the "rules of
nature" are pre-existant somehow, that they not only define nature but
actually cause it. If that's what you mean I'd like to hear your
further thoughts on the question.

Bill

## 12.2.2　特征提取

20News Groups Dataset包含大量的非字符符号，所以需要进行数据清洗，仅提取单词

In [17]:
import re
newsgroups_train = fetch_20newsgroups(subset='train')
sentences=[re.findall("[a-z]+",s.lower()) for s in newsgroups_train.data]

## 12.2.3　模型训练与验证

In [18]:
import gensim
from sklearn.datasets import fetch_20newsgroups
import re

model = gensim.models.Word2Vec(sentences, vector_size=200, window=5, min_count=1, workers=4)

In [19]:
keywords = ["user","password","email","name"]
for key in keywords:
    if key in model.wv:
        similar_words = model.wv.most_similar(key, topn=10)
        print(f"[{key}] most_similar: {similar_words}")
    else:
        print(f"The word '{key}' is not in the vocabulary.")

[user] most_similar: [('client', 0.7658645510673523), ('protocol', 0.7615214586257935), ('database', 0.754130482673645), ('server', 0.7438555955886841), ('users', 0.7414344549179077), ('ripem', 0.735898494720459), ('resource', 0.733748733997345), ('interface', 0.7276293039321899), ('sample', 0.7246660590171814), ('binary', 0.7239730954170227)]
[password] most_similar: [('binary', 0.7681770920753479), ('log', 0.763287365436554), ('username', 0.7619608044624329), ('database', 0.7516307234764099), ('format', 0.747757613658905), ('login', 0.7445029020309448), ('pgp', 0.7436643242835999), ('passphrase', 0.7398062348365784), ('variable', 0.7385547161102295), ('macro', 0.7350848317146301)]
[email] most_similar: [('replies', 0.8038066029548645), ('mail', 0.7871789336204529), ('contact', 0.7836530208587646), ('address', 0.7622296810150146), ('whjjm', 0.7537462711334229), ('request', 0.7195327877998352), ('send', 0.7158567905426025), ('subscribe', 0.7103419899940491), ('semd', 0.7089439034461975